In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%pip install segment-geospatial groundingdino-py localtileserver

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.3/82.3 kB 9.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.6/56.6 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 152.8/152.8 kB 16.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.8/46.8 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 3.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.8/87.8 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 104.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 100.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 108.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 515.7/515.7 kB 5.2 MB

In [ ]:
import numpy as np
import os
import cv2
from pycocotools import mask as cocomask
from tqdm import tqdm
from samgeo.text_sam import LangSAM

def calculate_mask_iou(mask1, mask2):
    mask1_bool = mask1.astype(bool)
    mask2_bool = mask2.astype(bool)

    intersection = np.sum(mask1_bool & mask2_bool)
    union = np.sum(mask1_bool | mask2_bool)

    if union == 0:
        return 0.0 # Avoid division by zero if both masks are empty
    return intersection / union

def make_masks_mutually_exclusive_and_prioritize_whole_object(masks, phrases, height, width):
    """
    Processes a list of SAM masks and their corresponding phrases to ensure mutual exclusivity
    while prioritizing larger, "whole object" masks. Overlapping regions of smaller masks
    are removed.

    Args:
        masks (list of np.ndarray): List of binary masks (boolean or uint8) from SAM.
                                   Each mask is typically of shape (height, width).
        phrases (list of str): Corresponding phrases/labels for each mask.
        height (int): The height of the original image.
        width (int): The width of the original image.

    Returns:
        tuple: (list of np.ndarray, list of str)
               - A list of processed binary masks, where no two masks overlap.
               - A list of corresponding phrases for the filtered masks.
    """
    if len(masks) == 0:
        return [], []

    # Store masks with their original phrase and calculated area
    mask_info = []
    for i, mask in enumerate(masks):
        mask_bool = mask.astype(bool)
        area = np.sum(mask_bool)
        if area > 0:
            mask_info.append({'mask': mask_bool, 'phrase': phrases[i], 'area': area})

    # Sort masks by area in descending order. Prioritizing larger masks
    mask_info.sort(key=lambda x: x['area'], reverse=True)

    final_masks = []
    final_phrases = []
    # Keeps track of all pixels already claimed by a bigger mask.
    covered_pixels = np.zeros((height, width), dtype=bool)

    for info in mask_info:
        current_mask = info['mask']
        current_phrase = info['phrase']

        exclusive_part = current_mask & (~covered_pixels)

        # We add this exclusive part to list only if it's substantial enough.
#----------------------------------Tunable value------------------------------------------------        
        if np.sum(exclusive_part) > 100:
#----------------------------------Tunable value------------------------------------------------
            final_masks.append(exclusive_part)
            final_phrases.append(current_phrase)
            covered_pixels = covered_pixels | exclusive_part

    return final_masks, final_phrases

def process_sequence(seq_dir, sam_model, text_prompt="car"):
    """
    Process all images in a sequence directory and save results in KITTI MOTS format.
    Simple IoU-based tracking also here.
    """
    seq_name = os.path.basename(seq_dir)
    # Output file path
    output_file = f"/content/drive/MyDrive/thesis project_data/results/{seq_name}.txt"

    img_paths = sorted([os.path.join(seq_dir, f)
                      for f in os.listdir(seq_dir) if f.endswith(".png")])

    # Set up for Tracking
    last_frame_object_info = []
    next_instance_id_map = {1: 1, 2: 1} # 1: car, 2: pedestrian

#-----------------Tunable IoU threshold for considering two masks a match for tracking----------
    iou_threshold = 0.3
#-----------------Tunable IoU threshold for considering two masks a match for tracking----------

    with open(output_file, "w") as results_file:
        for img_path in tqdm(img_paths, desc=f"Processing {seq_name}"):
            # Extract frame number from filename ('000000.png' -> 0)
            frame_num = int(os.path.splitext(os.path.basename(img_path))[0])

            image = cv2.imread(img_path)
            if image is None:
                print(f"Could not load image {img_path}. Skipping.")
                continue
            height, width = image.shape[:2]

#----------------------------------Tunable SAM Thresholds--------------------------------------
            masks_raw, _, phrases_raw, _ = sam_model.predict(
                img_path, text_prompt, 0.24, 0.24, return_results=True
            )
#----------------------------------Tunable SAM Thresholds--------------------------------------

            # Post-process masks to ensure mutual exclusivity and whole-object prioritization
            masks_processed, phrases_processed = make_masks_mutually_exclusive_and_prioritize_whole_object(
                masks_raw, phrases_raw, height, width
            )

            # Tracking :)
            current_frame_object_info = []
            matched_prev_ids = set()

            for i, current_mask in enumerate(masks_processed):
                current_phrase = phrases_processed[i]
                current_cls_id = 1 if current_phrase.lower() == "car" else 2 # Change here if more classes

                assigned_id = None
                best_iou = -1.0
                best_match_idx = -1

                # Try to find a match with objects from the last frame
                for prev_idx, prev_obj_info in enumerate(last_frame_object_info):
                    # Only consider previous objects that haven't been matched yet in this frame
                    if prev_obj_info['id'] in matched_prev_ids:
                        continue

                    # Ensure class consistency
                    if prev_obj_info['class_id'] != current_cls_id:
                        continue

                    iou = calculate_mask_iou(current_mask, prev_obj_info['mask'])

                    if iou > iou_threshold and iou > best_iou:
                        best_iou = iou
                        assigned_id = prev_obj_info['id']
                        best_match_idx = prev_idx

                if assigned_id is not None:
                    current_frame_object_info.append({
                        'mask': current_mask,
                        'phrase': current_phrase,
                        'id': assigned_id,
                        'class_id': current_cls_id
                    })
                    # Mark the matched previous object ID as used
                    matched_prev_ids.add(assigned_id)
                else:
                    # No match found, assign a new ID for this new object
                    new_instance_id = next_instance_id_map.get(current_cls_id, 1)
                    assigned_id = current_cls_id * 1000 + new_instance_id
                    next_instance_id_map[current_cls_id] = new_instance_id + 1

                    current_frame_object_info.append({
                        'mask': current_mask,
                        'phrase': current_phrase,
                        'id': assigned_id,
                        'class_id': current_cls_id
                    })
            # End Tracking

            # Generate KITTI MOTS format lines for this frame
            frame_lines = generate_kitti_lines(current_frame_object_info, frame_num, height, width)
            results_file.writelines(frame_lines)

            last_frame_object_info = current_frame_object_info # For next iter

def generate_kitti_lines(tracked_objects_info, frame_num, height, width):
    """
    Converts tracked object information (processed masks, phrases, and consistent IDs)
    into KITTI MOTS format lines.

    Args:
        tracked_objects_info (list of dict): List of dictionaries, each containing
                                            'mask', 'phrase', 'id', and 'class_id' for an object.
        frame_num (int): The current frame number.
        height (int): Image height.
        width (int): Image width.

    Returns:
        list: A list of strings, each representing a line in the KITTI MOTS format.
    """
    lines = []
    sorted_objects_info = sorted(tracked_objects_info, key=lambda x: get_bbox_center(x['mask'])[0])

    for obj_info in sorted_objects_info:
        mask = obj_info['mask']
        obj_id = obj_info['id']
        cls_id = obj_info['class_id']

        # RLE (Run-Length Encoding) of the mask, required by KITTI MOTS.
        rle = cocomask.encode(np.asfortranarray(mask.astype(np.uint8)))
        rle_str = rle['counts'].decode('utf-8')

        lines.append(f"{frame_num} {obj_id} {cls_id} {height} {width} {rle_str}\n")

    # "ignore region" mask
    all_masks_for_ignore = [info['mask'] for info in tracked_objects_info]
    ignore_mask = get_ignore_region(all_masks_for_ignore, height, width)
    rle = cocomask.encode(np.asfortranarray(ignore_mask.astype(np.uint8)))
    lines.append(f"{frame_num} 10000 10 {height} {width} {rle['counts'].decode('utf-8')}\n")

    return lines

def get_bbox_center(mask):

    y, x = np.where(mask)
    if len(x) == 0 or len(y) == 0:
        return (0, 0)
    return (x.mean(), y.mean())

def get_ignore_region(masks, height, width):
    """
    Generates a binary mask representing the 'ignore' region, i.e., all pixels
    not covered by any of the provided object masks.

    Args:
        masks (list of np.ndarray): A list of binary object masks.
        height (int): Image height.
        width (int): Image width.

    Returns:
        np.ndarray: A boolean mask where True indicates an ignore region.
    """
    if not masks:
        return np.ones((height, width), dtype=bool)

    combined_object_mask = np.zeros((height, width), dtype=bool)

    for mask in masks:
        combined_object_mask = np.logical_or(combined_object_mask, mask.astype(bool))

    return ~combined_object_mask


In [ ]:
sam = LangSAM(model_type="sam2-hiera-large")

final text_encoder_type: bert-base-uncased


In [ ]:
if __name__ == "__main__":
    # sam = LangSAM(model_type="sam2-hiera-large")
    testing_root = "/content/drive/MyDrive/thesis project_data/val" # changed for G-Drive
    for seq_name in sorted(os.listdir(testing_root)):
        seq_dir = os.path.join(testing_root, seq_name)
        if os.path.isdir(seq_dir):
            print(f"Processing sequence: {seq_name}")
            process_sequence(seq_dir, sam_model=sam, text_prompt="car")

Processing sequence: 0002


Processing 0002: 100%|██████████| 233/233 [05:00<00:00,  1.29s/it]


Processing sequence: 0006


Processing 0006:  89%|████████▉ | 241/270 [05:08<00:28,  1.00it/s]

No objects found in the image.


...

Processing 0014: 100%|██████████| 106/106 [02:21<00:00,  1.34s/it]


Processing sequence: 0016


Processing 0016: 100%|██████████| 209/209 [04:25<00:00,  1.27s/it]


Processing sequence: 0018


Processing 0018: 100%|██████████| 339/339 [07:18<00:00,  1.29s/it]
